In [1]:
! python -m pip install gurobipy


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Lettura file CSV
df_t = pd.read_csv("data/pc_trasporto.csv", index_col=0)
# df_t.index = df_t.index.str.strip()
# df_t.columns = df_t.columns.str.strip()

df_p = pd.read_csv("data/pc_produzione.csv", index_col=0)
# df_p.index = df_p.index.str.strip()
# df_p.columns = df_p.columns.str.strip()

# Definizione insiemi
I = df_t.index.drop("domanda").tolist()
J = df_t.columns.tolist()
A = df_p["capacita"].to_dict()
W = df_p["costo"].to_dict()
B = df_t.loc["domanda"].to_dict()
C = df_t.drop(index="domanda")

print("Stabilimenti:", I)
print("Banche:", J)
print("Capacità produttive:", A)
print("Domanda:", B)
print("Costi di produzione:", W)

# Creazione modello
m = gp.Model("pc")
m.setParam("OutputFlag", 0)

# Variables: x[i,j] = shipped from stabilimento i to banca j
x = m.addVars(I, J, vtype=GRB.INTEGER, lb=0, name="x")

# Definizione vincoli
for i in I:
    m.addConstr(gp.quicksum(x[i, j] for j in J) <= A[i], name=f"capacita_[{i}]")

for j in J:
    m.addConstr(gp.quicksum(x[i, j] for i in I) == B[j], name=f"domanda_[{j}]")

# Definizione O.f.
m.setObjective(
    gp.quicksum((W[i] + C.loc[i, j]) * x[i, j] for i in I for j in J), GRB.MINIMIZE
)

# Ottimizzazione
m.optimize()

# Stampa risultati
if m.status == GRB.OPTIMAL:
    print("\n=== RISULTATI ===")
    print(f"Valore funzione obiettivo: {m.objVal:.2f}\n")

    for i in I:
        for j in J:
            if x[i, j].X > 1e-6:  # mostra solo variabili positive
                print(f"Quantità spedita da {i} a {j}: {x[i, j].X:.0f}")

    print("\nCapacità utilizzate:")
    for i in I:
        used = sum(x[i, j].X for j in J)
        print(f"{i}: {used:.0f} / {A[i]}")

    print("\nDomanda soddisfatta:")
    for j in J:
        got = sum(x[i, j].X for i in I)
        print(f"{j}: {got:.0f} (domanda {B[j]:.0f})")
else:
    print("Nessuna soluzione ottimale trovata. Stato:", m.status)

Stabilimenti: ['it', 'cn', 'fr']
Banche: ['banca1', 'banca2', 'banca3', 'banca4', 'banca5']
Capacità produttive: {'it': 11000, 'cn': 20000, 'fr': 10000}
Domanda: {'banca1': 7100.0, 'banca2': 3400.0, 'banca3': 9700.0, 'banca4': 5200.0, 'banca5': 3050.0}
Costi di produzione: {'it': 220, 'cn': 180, 'fr': 200}
Set parameter Username
Academic license - for non-commercial use only - expires 2026-06-10

=== RISULTATI ===
Valore funzione obiettivo: 5592580.00

Quantità spedita da cn a banca1: 3850
Quantità spedita da cn a banca2: 3400
Quantità spedita da cn a banca3: 9700
Quantità spedita da cn a banca5: 3050
Quantità spedita da fr a banca1: 3250
Quantità spedita da fr a banca4: 5200

Capacità utilizzate:
it: 0 / 11000
cn: 20000 / 20000
fr: 8450 / 10000

Domanda soddisfatta:
banca1: 7100 (domanda 7100)
banca2: 3400 (domanda 3400)
banca3: 9700 (domanda 9700)
banca4: 5200 (domanda 5200)
banca5: 3050 (domanda 3050)
